In [26]:
import numpy as np

In [27]:
x_data = np.matrix([
#    Users  |     Movies     |    Movie Ratings   | Time | Last Movies Rated
#   A  B  C | TI  NH  SW  ST | TI   NH   SW   ST  |      | TI  NH  SW  ST
    [1, 0, 0,  1,  0,  0,  0,   0.3, 0.3, 0.3, 0,     13,   0,  0,  0,  0 ],
    [1, 0, 0,  0,  1,  0,  0,   0.3, 0.3, 0.3, 0,     14,   1,  0,  0,  0 ],
    [1, 0, 0,  0,  0,  1,  0,   0.3, 0.3, 0.3, 0,     16,   0,  1,  0,  0 ],
    [0, 1, 0,  0,  0,  1,  0,   0,   0,   0.5, 0.5,   5,    0,  0,  0,  0 ],
    [0, 1, 0,  0,  0,  0,  1,   0,   0,   0.5, 0.5,   8,    0,  0,  1,  0 ],
    [0, 0, 1,  1,  0,  0,  0,   0.5, 0,   0.5, 0,     9,    0,  0,  0,  0 ],
    [0, 0, 1,  0,  0,  1,  0,   0.5, 0,   0.5, 0,     12,   1,  0,  0,  0 ]
])

# ratings
y_data = np.array([5, 3, 1, 4, 5, 1, 5])

# Let's add an axis to make tensoflow happy.
y_data.shape += (1, )

In [28]:
import tensorflow as tf
n, p = x_data.shape

# number of latent factors
k = 5

# design matrix
X = tf.placeholder('float', shape=[n, p])
# target vector
y = tf.placeholder('float', shape=[n, 1])

# bias and weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([p]))

# interaction factors, randomly initialized 
V = tf.Variable(tf.random_normal([k, p], stddev=0.01))

# estimate of y, initialized to 0.
y_hat = tf.Variable(tf.zeros([n, 1]))

Calculates $w_o + \sum_{i = 1}^{p} w_i x_i$

In [29]:
linear_terms = tf.add(w0,
        	tf.reduce_sum(
                tf.multiply(W, X), 1, keep_dims=True))

Calculates $\frac{1}{2} \sum_{f = 1}^{k} ((\sum_{i}^{p} v_{if}x_i)^2 - \sum_{i = 1}^{p} v_{if}^2 x_i^2)$

In [30]:
interactions = (tf.multiply(0.5,
                tf.reduce_sum(
                    tf.sub(
                        tf.pow( tf.matmul(X, tf.transpose(V)), 2),
                        tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2)))),
                    1, keep_dims=True)))

In [31]:
y_hat = tf.add(linear_terms, interactions)

Learn the model by minimizing the sum of squares loss function. Add L2 regularization term to prevent overfitting. 

In [32]:
# L2 regularized sum of squares loss function over W and V
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_w, tf.pow(W, 2)),
                tf.multiply(lambda_v, tf.pow(V, 2)))))

error = tf.reduce_mean(tf.square(tf.sub(y, y_hat)))
loss = tf.add(error, l2_norm)

Train model by instantiate Optimizer object and minimize loss function. 

In [33]:
eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

In [34]:
# that's a lot of iterations
N_EPOCHS = 1000
# Launch the graph.
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(N_EPOCHS):
        indices = np.arange(n)
        np.random.shuffle(indices)
        x_data, y_data = x_data[indices], y_data[indices]
        sess.run(optimizer, feed_dict={X: x_data, y: y_data})

    print('MSE: ', sess.run(error, feed_dict={X: x_data, y: y_data}))
    #print('Loss (regularized error):', sess.run(cost, feed_dict={X: x_data, y: y_data}))
    print('Loss (regularized error):', sess.run(loss, feed_dict={X: x_data, y: y_data}))
    print('Predictions:', sess.run(y_hat, feed_dict={X: x_data, y: y_data}))
    print('Learnt weights:', sess.run(W, feed_dict={X: x_data, y: y_data}))
    print('Learnt factors:', sess.run(V, feed_dict={X: x_data, y: y_data}))

MSE:  9.56202e-05
Loss (regularized error): 0.00328812
Predictions: [[ 5.00942278]
 [ 1.00716376]
 [ 4.00393629]
 [ 5.00835752]
 [ 1.01295185]
 [ 3.01246548]
 [ 5.01098871]]
Learnt weights: [ 0.13115798  0.19350205 -0.11888152 -0.01963052 -0.06807373  0.20083955
  0.08878157 -0.01278166  0.07432894  0.07578647  0.14581633  0.10008761
  0.16054869 -0.18174787  0.08878157  0.        ]
Learnt factors: [[-0.1192272  -0.26376137  0.1061745  -0.13125207  0.26142853 -0.2032536
  -0.0850707  -0.01416051 -0.06831901 -0.09288774 -0.23749761 -0.25603428
  -0.24092621  0.38513494 -0.08550519  0.00335897]
 [ 0.09893265  0.23286533 -0.14457522  0.11698433 -0.27972937  0.19220932
   0.04177043 -0.01598943  0.05023088  0.046404    0.21351211  0.276452
   0.22398117 -0.42053214  0.04309372  0.00078712]
 [-0.1039304  -0.27868834  0.14098421 -0.12209997  0.28104118 -0.20802394
  -0.07553737  0.0118631  -0.04986483 -0.09383552 -0.23956515 -0.27726933
  -0.24224874  0.42145225 -0.0744479  -0.00319736]
 [ 0